In [10]:
import pandas as pd
import csv

In [11]:
nba = pd.read_csv("all_seasons.csv")

In [12]:
# Gather necessary columns
college_nba_pipeline = nba[["player_name", "team_abbreviation", "college"]]

In [13]:
college_nba_pipeline

,player_name,team_abbreviation,college
0,Randy Livingston,HOU,Louisiana State
1,Gaylon Nickerson,WAS,Northwestern Oklahoma
2,George Lynch,VAN,North Carolina
3,George McCloud,LAL,Florida State
4,George Zidek,DEN,UCLA
...,...,...,...
12839,Joel Embiid,PHI,Kansas
12840,John Butler Jr.,POR,Florida State
12841,John Collins,ATL,Wake Forest
12842,Jericho Sims,NYK,Texas


In [14]:
def create_adjacency_list(data):
    # Create an empty dictionary to store the adjacency list
    adjacency_list = {}

    # Iterate over each row in the data
    for index, row in data.iterrows():
        college = row['college']
        team = row['team_abbreviation']

        # If college is not in the adjacency list, add it with an empty list
        if college not in adjacency_list:
            adjacency_list[college] = []

        # Append the team to the college's list if it's not already there
        if team not in adjacency_list[college]:
            adjacency_list[college].append(team)

    return adjacency_list

In [15]:
adj_list = create_adjacency_list(college_nba_pipeline)

In [16]:
with open('dict.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in adj_list.items():
       writer.writerow([key, value])